In [1]:
!pip install -U "optimum>=1.20.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 30.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 35.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 28.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install torch transformers accelerate auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 67.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 62.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True  # Qwenモデルには必要
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True  # Qwenモデルには必要
)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/3.48G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:5055: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [5]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)

README.md:   0%|          | 0.00/8.37k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/115k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
!pip install jsonlines


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [8]:
import torch
import jsonlines

def generate_text(input_text):
    # Qwen公式のQuickstart形式でapply_chat_templateを使用
    messages = [
        {"role": "system", "content": "あなたは役立つアシスタントです。ユーザーの質問に回答し、指示に従ってください。"},
        {"role": "user", "content": input_text}
    ]

    # テンプレート適用（ChatML形式への変換）
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # トークナイズ
    model_inputs = tokenizer([prompt_text], return_tensors="pt").to(model.device)

    # 再現性確保のためのシード設定（必要に応じて変更）
    seed = 42
    torch.manual_seed(seed)

    # ハイパーパラメータはモデルの推奨設定に応じて調整可能
    tokens = model.generate(
        **model_inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    # 入力トークン分を除去し、生成部分のみ抽出
    output_tokens = tokens[0][model_inputs.input_ids.shape[1]:]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # データセットを読み、生成結果を保存
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for i, obj in enumerate(reader):
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            writer.write({"pred": generated_text})

            print("========================================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print()
            
            # 定期的にキャッシュをクリア（例：10件ごと）
            if i % 10 == 0:
                torch.cuda.empty_cache()

# 処理対象ファイルを指定し実行
input_dataset = '../assets/elyza_tasks_100/dataset.jsonl'
output_predictions = '../assets/elyza_tasks_100/preds.jsonl'
process_dataset(input_dataset, output_predictions)

Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 1. 目標を再評価する: あなたの目標が自分自身にとって重要で達成可能であることを確認します。これにより、あなたが行っている仕事が自分自身の長期的なキャリア目標とどのようにつながっているかを理解することができます。

2. ボスや同僚との対話: ボスや同僚と定期的に会話し、自分の進捗状況や問題点を共有します。彼らからのフィードバックやサポートは、あなたがモチベーションを取り戻すのに役立ちます。

3. 新しいスキルの学習: 自分自身のスキルセットを拡張することを考えてみてください。新しいスキルを学ぶことで、新たな視野を開くことができ、それがあなたの仕事に対する見方を変え、さらにやりがいを感じるかもしれません。

4. ブレイクタイムを取り入れる: 必要に応じて休息をとることは非常に重要です。リフレッシュすることで、新しいエネルギーと情熱を仕事にもたらすことができます。

5. ポジティブな思考を維持する: 負の感情や考えにとらわれず、常にポジティブな視点を持つように心掛けることが重要です。これは、困難な時期でも前向きに取り組む助けとなります。

Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A. タイトル: 「海辺の出会い」

春の訪れとともに、森のクマの子、ココはいつもとは違う冒険に出ることを決意しました。彼はこれまで一度も訪れたことのない海辺を目指して旅立ちました。

彼が海辺に到着したとき、彼の目は驚きで輝きました。砂浜にはアザラシのプーがいました。プーは波の音に合わせて踊り、その様子はとても楽しそうでした。ココはプーに近づき、彼の新しい友人になることを望みました。

「こんにちは、私はココです」とココは挨拶しました。プーは頭を上げ、ココを見ました。「こんにちは、ココ。私はプーです。森の住民として、あなたは珍しい訪問者ですね。」

二人はすぐに打ち解け、一日中一緒に遊びました。プーはココに海の秘密を教えてくれました。彼はココに海の生物や海の動きについて教えてくれました。ココはプーの話を聞きながら、海の不思議さを感じました。

しかし、太陽は徐々に西に傾き始め、夜が訪れようとしていました。ココは家に帰りたくなったので、彼はプーに別れ

In [11]:
with open('/content/dataset.jsonl', 'r') as f:
    data_length = sum(1 for _ in f)
print(data_length)

100


In [5]:
with open('/content/preds.jsonl', 'r') as f:
    data_length = sum(1 for _ in f)
print(data_length)

FileNotFoundError: [Errno 2] No such file or directory: '/content/preds.jsonl'

In [2]:
!pip install google


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
from IPython.display import FileLink
FileLink('/content/preds.jsonl')

/content/preds.jsonl